In [1]:
import pyspark
import pandas

El servicio metereológico registra datos del tiempo para todas las ciudades donde posee una base de medición. Esta información se encuentra en dos RDD. En el primero se tiene informacion de las bases de medicion (id_base, nombre, pcia, ciudad, lat, lon). El segundo posee información sobre las mediciones en sí: (timestamp, id_base, temperatura, humedad, presión, dirección viente, velocidad viento). 
Se desea obtener un reporte para las bases de la Pcia de Bs As. El mismo debe informar los ID y nombre de bases de medición que hayan registrado una variacion de temperatura promedio mensual mayor al 30% en el año 2018 (dada la temperatura promedio de un mes, esta se debe comparar con el promedio del mes anterior, para determinar la variacion porcentual)

In [7]:
# (id_base,nombre,pcia,ciudad,lat,lon)
bases = [
    (1, 'base1', 'Buenos Aires', 'San Fernando', -1432, 93482),
    (2, 'base2', 'Mendoza', 'San Rafael', -32432, 43243),
    (3, 'base3', 'Mendoza', 'Las Leñas', -32313, 354345),
    (4, 'base4', 'Buenos Aires', 'CABA', -123123, 432423),
    (5, 'base5', 'Neuquen', 'Bariloche', -634534, 564563),
    (6, 'base6', 'Buenos Aires', 'Merlo', -453453,57454),
    (7, 'base7', 'Buenos Aires', 'Avellaneda', -2493284, -3428234),
    (8, 'base8', 'Buenos Aires', 'Tigre', -32432, 654654),
    (9, 'base9', 'Buenos Aires', 'Quilmes', -98324, -3284723),
    (10, 'base10', 'Misiones', 'San Ignacio', -654645, 564345)
]

# (timestamp, id_base, temperatura, humedad, presion, direccion viento, velocidad viento)
mediciones = [
    ('2018-03-04', 1, 25, 96, 300, 'NO', 47),
    ('2017-12-03', 1, 25, 96, 300, 'NO', 47),
    ('2018-04-05', 1, 15, 96, 300, 'NO', 47),
    ('2018-05-12', 1, 18, 96, 300, 'NO', 47),
    ('2018-05-21', 1, 23, 96, 300, 'NO', 47),
    ('2018-01-04', 1, 38, 96, 300, 'NO', 47),
    ('2018-02-29', 1, 28, 96, 300, 'NO', 47),
    ('2018-06-14', 1, 9, 96, 300, 'NO', 47),
    ('2018-07-12', 1, 5, 96, 300, 'NO', 47),
    ('2018-08-31', 1, 19, 96, 300, 'NO', 47),
    ('2018-09-14', 1, 20, 96, 300, 'NO', 47),
    ('2019-03-12', 1, 23, 96, 300, 'NO', 47),
    ('2019-04-15', 1, 16, 96, 300, 'NO', 47),
    ('2018-07-12', 2, 5, 96, 300, 'NO', 47),
    ('2018-08-31', 2, 19, 96, 300, 'NO', 47),
    ('2018-09-14', 4, 26, 96, 300, 'NO', 47),
    ('2018-08-12', 4, 13, 96, 300, 'NO', 47),
    ('2019-04-15', 4, 16, 96, 300, 'NO', 47)
]

In [109]:
basesRdd = sc.parallelize(bases)
medicionesRdd = sc.parallelize(mediciones)

In [110]:
#primero filtro bases de buenos aires
basesRdd = basesRdd.filter(lambda x: x[2] == 'Buenos Aires')
basesRdd.collect()

[(1, 'base1', 'Buenos Aires', 'San Fernando', -1432, 93482),
 (4, 'base4', 'Buenos Aires', 'CABA', -123123, 432423),
 (6, 'base6', 'Buenos Aires', 'Merlo', -453453, 57454),
 (7, 'base7', 'Buenos Aires', 'Avellaneda', -2493284, -3428234),
 (8, 'base8', 'Buenos Aires', 'Tigre', -32432, 654654),
 (9, 'base9', 'Buenos Aires', 'Quilmes', -98324, -3284723)]

In [111]:
#filtro mediciones de 2018
medicionesRdd = medicionesRdd.filter(lambda x: '2018' in x[0])
medicionesRdd.collect()

[('2018-03-04', 1, 25, 96, 300, 'NO', 47),
 ('2018-04-05', 1, 15, 96, 300, 'NO', 47),
 ('2018-05-12', 1, 18, 96, 300, 'NO', 47),
 ('2018-05-21', 1, 23, 96, 300, 'NO', 47),
 ('2018-01-04', 1, 38, 96, 300, 'NO', 47),
 ('2018-02-29', 1, 28, 96, 300, 'NO', 47),
 ('2018-06-14', 1, 9, 96, 300, 'NO', 47),
 ('2018-07-12', 1, 5, 96, 300, 'NO', 47),
 ('2018-08-31', 1, 19, 96, 300, 'NO', 47),
 ('2018-09-14', 1, 20, 96, 300, 'NO', 47),
 ('2018-07-12', 2, 5, 96, 300, 'NO', 47),
 ('2018-08-31', 2, 19, 96, 300, 'NO', 47),
 ('2018-09-14', 4, 26, 96, 300, 'NO', 47),
 ('2018-08-12', 4, 13, 96, 300, 'NO', 47)]

In [112]:
# mapeo ambos rdds
basesRdd = basesRdd.map(lambda x: (x[0], x[1]))
medicionesRdd = medicionesRdd.map(lambda x: (x[1], (x[0],x[2])))
# joineo ahora asi me quedan las mediciones de buenos aires
medsRdd = basesRdd.join(medicionesRdd)
medsRdd.collect()

[(1, ('base1', ('2018-03-04', 25))),
 (1, ('base1', ('2018-04-05', 15))),
 (1, ('base1', ('2018-05-12', 18))),
 (1, ('base1', ('2018-05-21', 23))),
 (1, ('base1', ('2018-01-04', 38))),
 (1, ('base1', ('2018-02-29', 28))),
 (1, ('base1', ('2018-06-14', 9))),
 (1, ('base1', ('2018-07-12', 5))),
 (1, ('base1', ('2018-08-31', 19))),
 (1, ('base1', ('2018-09-14', 20))),
 (4, ('base4', ('2018-09-14', 26))),
 (4, ('base4', ('2018-08-12', 13)))]

In [113]:
# calculo la temperatura promedio por mes, para eso pongo como clave el mes y año (esto en realidad lo tendria
# que haber hecho en el map anterior)
medsRdd = medsRdd.map(lambda x: ((x[1][1][0].split('-')[0], x[1][1][0].split('-')[1],x[0],x[1][0]),(x[1][1][1], 1)))
medsRdd.collect()

[(('2018', '03', 1, 'base1'), (25, 1)),
 (('2018', '04', 1, 'base1'), (15, 1)),
 (('2018', '05', 1, 'base1'), (18, 1)),
 (('2018', '05', 1, 'base1'), (23, 1)),
 (('2018', '01', 1, 'base1'), (38, 1)),
 (('2018', '02', 1, 'base1'), (28, 1)),
 (('2018', '06', 1, 'base1'), (9, 1)),
 (('2018', '07', 1, 'base1'), (5, 1)),
 (('2018', '08', 1, 'base1'), (19, 1)),
 (('2018', '09', 1, 'base1'), (20, 1)),
 (('2018', '09', 4, 'base4'), (26, 1)),
 (('2018', '08', 4, 'base4'), (13, 1))]

In [134]:
medsRdd = medsRdd.reduceByKey(lambda x,y: ((x[0]+y[0])/(x[1]+y[1]))) #esto(promedio) mejor hacerlo en dos pasos!
medsRdd = medsRdd.map(lambda x: (x[0], x[1][0]/x[1][1]) if ((type(x[1])==tuple)) else (x[0],x[1])).sortByKey()
medsRdd.collect()

[(('2018', '01', 1, 'base1'), 38.0),
 (('2018', '02', 1, 'base1'), 28.0),
 (('2018', '03', 1, 'base1'), 25.0),
 (('2018', '04', 1, 'base1'), 15.0),
 (('2018', '05', 1, 'base1'), 20.5),
 (('2018', '06', 1, 'base1'), 9.0),
 (('2018', '07', 1, 'base1'), 5.0),
 (('2018', '08', 1, 'base1'), 19.0),
 (('2018', '08', 4, 'base4'), 13.0),
 (('2018', '09', 1, 'base1'), 20.0),
 (('2018', '09', 4, 'base4'), 26.0)]

In [149]:
def comparar(x):
    tempAnterior = 0
    newList = []
    for registro in x[1]:
        if ((abs(registro[0]-tempAnterior)/registro[0]) > 0.3):
                newReg = ((abs(registro[0]-tempAnterior)/registro[0]), registro[3])
                newList.append(newReg)
        tempAnterior = registro[0]
        #return newList
    #x[1] = newList
    return (x[0], newList)
    
        

#medsRdd.map(lambda x: (x[0][2], (x[1],x[0][0],x[0][1],x[0][3]))).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
medsRdd.map(lambda x: (x[0][2], (x[1],x[0][0],x[0][1],x[0][3]))).groupByKey().map(lambda x: (x[0], list(x[1])))\
.map(comparar)\
.collect()

[(1,
  [(1.0, 'base1'),
   (0.35714285714285715, 'base1'),
   (0.6666666666666666, 'base1'),
   (1.2777777777777777, 'base1'),
   (0.8, 'base1'),
   (0.7368421052631579, 'base1')]),
 (4, [(1.0, 'base4'), (0.5, 'base4')])]

In [150]:
#def mes_anterior(x):
#    if (x[0][1] == '12'):
#        anioAnterior = str(int(x[0][0])-1)
#    else:
#        anioAnterior = x[0][0]
#    if (x[0][1] == '01'):
#        mesAnterior = '12'
#    else:
#        mesAnterior = str(int(x[0][1])-1)
#    if (type(x[1]) == tuple) | (type(x[1]) == list):
#        return ((x[0][0], x[0][1], x[0][2], x[0][3]),(x[1][0]/x[1][1], mesAnterior, anioAnterior))
#    else:
#        return ((x[0][0], x[0][1], x[0][2], x[0][3]),(x[1], mesAnterior, anioAnterior))

# a cada registro le guardo el mes anterior
#medsRdd.map(lambda x: (x[2], (x[])))